## TO DOs (internal)
* add dual values to results

### Reminder (Before we start)
Change the kernel at the top and set it to "urbs" to be able to run this script.

# Example 1: Electricity supply of an island

## Learning objectives
* Translate a mathematical optimization problem into a pyomo ConcreteModel/AbstractModel
* Recognize the basic structure of an optimization model
* Report the results of pyomo in different formats

## Mathematical formulation

We start with a simple example. Let's assume we have a gas power plant ($P_{gas}$ = 100 MW) and a biomass power plant ($P_{bm}$ = 30 MW) supplying an island. The cost of supplying 1 MWh of electricity using the gas power plant is EUR 50, whereas the cost of using biomass is 25 EUR/MWh.
We would like to minimize the cost of operating the system for a given demand of electricity $d(t)$.

$$\min \quad 50s_{gas}(t) + 25s_{bm}(t)$$
$$s.t. \quad s_{gas}(t) + s_{bm}(t) \geq d(t)$$

The supply from the power plant is non-negative:
$$s_{gas}(t), s_{bm}(t) \geq 0$$

It cannot exceed the capacity of the power plants:
$$s_{gas}(t) \leq 100$$
$$s_{bm}(t) \leq 30$$

Further, we define the demand as follows:
$$d(t) = [60, 100, 120, 80, 30]$$

### <span style="color:blue">Task</span>
Try to solve this problem with pen and paper!

## Formulation as a pyomo ConcreteModel

We could solve this problem using a pyomo ConcreteModel:

In [1]:
import pyomo.environ as pyo
from pyomo.environ import *

model = pyo.ConcreteModel()
model.name = "Example1"

# Variables
model.s = pyo.Var(RangeSet(1, 5), RangeSet(1, 2), domain=pyo.NonNegativeReals)

# Objective function
model.OBJ = pyo.Objective(expr=50*model.s[1,1] + 25*model.s[1,2] +\
                               50*model.s[2,1] + 25*model.s[2,2] +\
                               50*model.s[3,1] + 25*model.s[3,2] +\
                               50*model.s[4,1] + 25*model.s[4,2] +\
                               50*model.s[5,1] + 25*model.s[5,2])

# Constraints
model.ConstraintGasCap1 = pyo.Constraint(expr = model.s[1, 1] <= 100)
model.ConstraintGasCap2 = pyo.Constraint(expr = model.s[2, 1] <= 100)
model.ConstraintGasCap3 = pyo.Constraint(expr = model.s[3, 1] <= 100)
model.ConstraintGasCap4 = pyo.Constraint(expr = model.s[4, 1] <= 100)
model.ConstraintGasCap5 = pyo.Constraint(expr = model.s[5, 1] <= 100)

model.ConstraintBiomassCap1 = pyo.Constraint(expr = model.s[1, 2] <= 30)
model.ConstraintBiomassCap2 = pyo.Constraint(expr = model.s[2, 2] <= 30)
model.ConstraintBiomassCap3 = pyo.Constraint(expr = model.s[3, 2] <= 30)
model.ConstraintBiomassCap4 = pyo.Constraint(expr = model.s[4, 2] <= 30)
model.ConstraintBiomassCap5 = pyo.Constraint(expr = model.s[5, 2] <= 30)

model.ConstraintDem1 = pyo.Constraint(expr = model.s[1,1] + model.s[1,2] >= 60)
model.ConstraintDem2 = pyo.Constraint(expr = model.s[2,1] + model.s[2,2] >= 100)
model.ConstraintDem3 = pyo.Constraint(expr = model.s[3,1] + model.s[3,2] >= 120)
model.ConstraintDem4 = pyo.Constraint(expr = model.s[4,1] + model.s[4,2] >= 80)
model.ConstraintDem5 = pyo.Constraint(expr = model.s[5,1] + model.s[5,2] >= 30)

In [3]:
# Write the LP mathematical problem that is solved to a file (optional)
# Here, we are reporting the model itself, not its solution
model.write("Example1a.lp")

('Example1b.lp', 2233384673912)

### <span style="color:blue">Task</span>
Open the file "Example1a.lp" with a text editor. Can you recognize the variables and constraints?

In [5]:
# Try this now
model.write("Example1b.lp", io_options={'symbolic_solver_labels': True})

('Example1b.lp', 2233384675816)

In [6]:
opt = pyo.SolverFactory('glpk')
results = opt.solve(model)
# First way of reporting the solution
results

{'Problem': [{'Name': 'unknown', 'Lower bound': 15750.0, 'Upper bound': 15750.0, 'Number of objectives': 1, 'Number of constraints': 16, 'Number of variables': 11, 'Number of nonzeros': 21, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.03581404685974121}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [12]:
print(results.Problem)


- Name: unknown
  Lower bound: 15750.0
  Upper bound: 15750.0
  Number of objectives: 1
  Number of constraints: 16
  Number of variables: 11
  Number of nonzeros: 21
  Sense: minimize



For more on solver status and termination conditions:
http://www.pyomo.org/blog/2015/1/8/accessing-solver

In [13]:
model.display()

Model Example1

  Variables:
    s : Size=10, Index=s_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (1, 1) :     0 :  30.0 :  None : False : False : NonNegativeReals
        (1, 2) :     0 :  30.0 :  None : False : False : NonNegativeReals
        (2, 1) :     0 :  70.0 :  None : False : False : NonNegativeReals
        (2, 2) :     0 :  30.0 :  None : False : False : NonNegativeReals
        (3, 1) :     0 :  90.0 :  None : False : False : NonNegativeReals
        (3, 2) :     0 :  30.0 :  None : False : False : NonNegativeReals
        (4, 1) :     0 :  50.0 :  None : False : False : NonNegativeReals
        (4, 2) :     0 :  30.0 :  None : False : False : NonNegativeReals
        (5, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (5, 2) :     0 :  30.0 :  None : False : False : NonNegativeReals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 15750.0

  Constraints:
 

### <span style="color:blue">Task</span>
1. Try to comment one or multiple constraints. What happens?
2. Try to maximize instead of minimizing the costs. (Tip: add the option 'sense=pyo.maximize' into the objective function)
3. How easy is it to add another power plant? Another time step?

## Formulation as a pyomo AbstractModel

One way to add flexibility is to write the problem abstractly. For example, the following equations represent a linear program (LP) to find optimal values for the vector $x$ (in our case, the hourly supply from the power plants) with parameters $c_j$ (costs), $a_{i,j}$ and $b_i$ (constraints):

$$ \begin{array}{lll} \min & \sum_{j=1}^n c_j x_{j,t} & \\
s.t. & \sum_{j=1}^n a_{i,j} x_{j,t} \geq b_{i,t} & \forall i = 1 \ldots m\\ & x_{j,t} \geq 0 & \forall j = 1 \ldots n
\end{array} $$ 

For that, there is the pyomo class AbstractModel:

In [93]:
from pyomo.environ import *

model = AbstractModel()

# Sets
model.I = Set() # we could define the dimensions, or let pyomo determine them from the data
model.J = Set()
model.T = Set()

# Parameters
model.a = Param(model.I, model.J)
model.b = Param(model.I, model.T)
model.c = Param(model.J)

# Variables
model.x = Var(model.J, model.T, domain=NonNegativeReals) # the variable is indexed by the set J

# Objective function
def obj_expression(model):
    sigma = 0
    for t in model.T:
        for j in model.J:
            sigma = sigma + model.c[j] * model.x[(j, t)]
    return sigma

model.OBJ = Objective(rule=obj_expression)

# Constraints
def ax_constraint_rule(model, i, t):
    # return the expression for the constraint for i
    return sum(model.a[i,j] * model.x[j, t] for j in model.J) >= model.b[i, t]

model.AxbConstraint = Constraint(model.I, model.T, rule=ax_constraint_rule) # this creates one constraint for each member of the set model.I

By running the code, we create an abstract model. Now we need to create an instance of it:

In [94]:
# We can create an instance without filling it with data
instance = model.create_instance()
instance.pprint()

7 Set Declarations
    AxbConstraint_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :    -- :    I*T :    0 :      {}
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :    -- :    Any :    0 :      {}
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :    -- :    Any :    0 :      {}
    T : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :    -- :    Any :    0 :      {}
    a_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :    -- :    I*J :    0 :      {}
    b_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :    -- :    I*T :    0 :      {}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :    -- :    J*T : 

In [95]:
# We can load data from a file (written in AMPL format)
data = DataPortal()
data.load(filename='abstract2.dat')
# You can view the defined sets and parameters
list(data.keys())

['I', 'J', 'T', 'c', 'a', 'b']

In [96]:
# We can create an instance that is filled with input data
instance = model.create_instance(data)
instance.pprint()

7 Set Declarations
    AxbConstraint_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*T :   15 : {('GasCap', 't1'), ('GasCap', 't2'), ('GasCap', 't3'), ('GasCap', 't4'), ('GasCap', 't5'), ('BiomassCap', 't1'), ('BiomassCap', 't2'), ('BiomassCap', 't3'), ('BiomassCap', 't4'), ('BiomassCap', 't5'), ('Dem', 't1'), ('Dem', 't2'), ('Dem', 't3'), ('Dem', 't4'), ('Dem', 't5')}
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'GasCap', 'BiomassCap', 'Dem'}
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'Gas', 'Biomass'}
    T : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'t1', 't2', 't3', 't4', 't5'}
    a_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Siz

In [97]:
opt = SolverFactory('glpk')
opt.solve(instance) 

{'Problem': [{'Name': 'unknown', 'Lower bound': 15750.0, 'Upper bound': 15750.0, 'Number of objectives': 1, 'Number of constraints': 16, 'Number of variables': 11, 'Number of nonzeros': 21, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.02132582664489746}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [98]:
instance.display()

Model unknown

  Variables:
    x : Size=10, Index=x_index
        Key               : Lower : Value : Upper : Fixed : Stale : Domain
        ('Biomass', 't1') :     0 :  30.0 :  None : False : False : NonNegativeReals
        ('Biomass', 't2') :     0 :  30.0 :  None : False : False : NonNegativeReals
        ('Biomass', 't3') :     0 :  30.0 :  None : False : False : NonNegativeReals
        ('Biomass', 't4') :     0 :  30.0 :  None : False : False : NonNegativeReals
        ('Biomass', 't5') :     0 :  30.0 :  None : False : False : NonNegativeReals
            ('Gas', 't1') :     0 :  30.0 :  None : False : False : NonNegativeReals
            ('Gas', 't2') :     0 :  70.0 :  None : False : False : NonNegativeReals
            ('Gas', 't3') :     0 :  90.0 :  None : False : False : NonNegativeReals
            ('Gas', 't4') :     0 :  50.0 :  None : False : False : NonNegativeReals
            ('Gas', 't5') :     0 :   0.0 :  None : False : False : NonNegativeReals

  Objectives:
 

### Tasks
1. Set the demand in the last time step to 140. What happens?
2. Add a third technology, PV, with zero running costs and with varying upper bounds for every time step.
3. How easy is it to add another power plant? Another time step?